In [ ]:
# konlpy 패키지 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 필요한 패키지들을 임포트
import pandas as pd
from konlpy.tag import Okt

In [ ]:
okt = Okt()


In [ ]:
# 데이터셋의 상위 10개 샘플 출력
reviews_df = pd.read_csv('https://raw.githubusercontent.com/leejin-kyu/logistic_sentiment/main/dataset.csv',encoding='cp949')

In [ ]:
reviews_df.head(10)

,Unnamed: 0,content,score,time
0,1,야놀자 믿고 가는거야~떠나보는거야~~^^,5,2017-04-29
1,2,좋아요,5,2019-05-21
2,3,감사합니다,5,2018-02-27
3,4,초초초쵸특가야놀쟈!,5,2018-08-03
4,5,굿,5,2016-07-09
5,6,테마샾에 이런 앱 광고해서 깔라고 하는거 고쳐주엇음 하네요 예쁜 테마 까려다가 이런...,1,2017-07-16
6,7,빠른 예약 및 결재도 간편해서 좋아요.,5,2017-05-07
7,8,좋아요,5,2019-10-24
8,9,중요한순간잘이용했습니다.,5,2018-08-20
9,10,좋아요,5,2019-11-29


In [ ]:
reviews_df = reviews_df.drop('Unnamed: 0', axis=1)
reviews_df.head(10)

,content,score,time
0,야놀자 믿고 가는거야~떠나보는거야~~^^,5,2017-04-29
1,좋아요,5,2019-05-21
2,감사합니다,5,2018-02-27
3,초초초쵸특가야놀쟈!,5,2018-08-03
4,굿,5,2016-07-09
5,테마샾에 이런 앱 광고해서 깔라고 하는거 고쳐주엇음 하네요 예쁜 테마 까려다가 이런...,1,2017-07-16
6,빠른 예약 및 결재도 간편해서 좋아요.,5,2017-05-07
7,좋아요,5,2019-10-24
8,중요한순간잘이용했습니다.,5,2018-08-20
9,좋아요,5,2019-11-29


In [ ]:
def tokenizer(text):
    morph = okt.pos(text)  # 품사정보를 추출해서 morph에 저장
    words = []  # 형태소 분석 후 저장될 리스트 변수
    for word, tag in morph:  # 품사정보가 ('A','B') 형태로 들어온다고 가정
        if tag in ['Adjective']:  # 명사,  형용사, 부사, 동사 품사 선택 'Noun', 'Adjective', 'Adverb', 'Verb'
            if len(word) > 1:
                words.append(word)
    return words

In [ ]:
reviews_df= reviews_df.astype('str') #astype은 자료형 변환 함수, 전처리에 용이하게 str로 변환

In [ ]:
reviews_df['content'] = reviews_df['content'].apply(tokenizer) # data의 total 컬럼에 형태소 추출함수인 tokenizer함수를 적용함

In [ ]:
stop_words = ['해서', '하지', '하는']  # 불용어 리스트
def remove_stopwords(text):
    for stop_word in stop_words:  # 불용어 리스트의 각 단어에 대해 반복
        text = text.replace(stop_word, '')  # 텍스트 내의 해당 불용어를 빈 문자열로 대체
    return text

reviews_df['content'] = reviews_df['content'].apply(remove_stopwords)

In [ ]:
reviews_df = reviews_df[reviews_df['score'] == '1']  # score 칼럼이 1인 데이터만 필터링


In [ ]:
reviews_df= reviews_df.astype('str') #astype은 자료형 변환 함수, 전처리에 용이하게 str로 변환

In [ ]:
#빈도도
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,ngram_range=(1,1)) #(옵션) 빈도순으로 정렬해서 가장 많이 나온 단어 1000단어까지를 TDM에 포함시킨다.
tdm=cv.fit_transform(reviews_df.content) # 단어 문서 행렬 생성, cv.fit_transform()안에 분석할 단어 문서 행렬을 넣어주면 됨. fit은 분석하고자 하는 데이터에 맞게 transform 변환을 하는데 dtm 방식으로 변환함.
word_count_tf=pd.DataFrame({'단어':cv.get_feature_names_out(),'빈도': tdm.sum(axis=0).flat}) #tdm.sum(axis=0)가 행렬형태이기때문에 flat으로 바꿔줌.
word_count_tf=word_count_tf.sort_values('빈도', ascending=False) #단어를 빈도를 기준으로 정렬함
word_count_tf=word_count_tf.reset_index(drop=True)
word_count_tf.index=word_count_tf.index+1
word_count_tf.to_csv(f'/content/빈도.csv',encoding='cp949') #csv 파일로 저장하기



In [ ]:
from wordcloud import WordCloud
import urllib.request
font_path = 'malgun.ttf'
font_url = 'https://github.com/leejin-kyu/logistic_sentiment/raw/main/malgun.ttf'
urllib.request.urlretrieve(font_url, font_path)  #  파일 다운로드
wc = WordCloud(background_color='white', max_words=100, width=500, height=500, font_path=font_path)
count_dic = dict(zip(word_count_tf['단어'], word_count_tf['빈도']))
cloud = wc.fit_words(count_dic)
cloud.to_image()